In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent, AgentState
from langchain.tools import tool, ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langchain_community.utilities import SQLDatabase
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
from langchain_mcp_adapters.client import MultiServerMCPClient
from tavily import TavilyClient
from typing import Dict, Any


class WeddingState(AgentState):
    origin: str
    destination: str
    date: str
    guest_count: str
    genre: str

## Create the subagents and their tools

In [ ]:
MODEL_NAME = "gemini-2.5-flash-lite"

tavily_client = TavilyClient()
db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

model = init_chat_model(model=MODEL_NAME, model_provider="google_genai")

config = {"configurable": {"thread_id": "1"}}

# Connect to Kiwi MCP server for flight search
kiwi_client = MultiServerMCPClient(
    {
        "travel_server": {
            "transport": "streamable_http",
            "url": "https://mcp.kiwi.com",
        }
    }
)
flight_tools = await kiwi_client.get_tools()

@tool
def venue_search(query: str) -> Dict[str, Any]:
    """Search the web for wedding venue information"""
    return tavily_client.search(query)

@tool
def playlist_query(query: str) -> str:
    """Query the Chinook music database using SQL to find tracks, playlists, genres, and artists for wedding playlists."""
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

flight_search_subagent = create_agent(
    model=model,
    tools=flight_tools,
    system_prompt=(
        "You are a travel agent. Search for flights to the desired destination wedding location. "
        "You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria: "
        "Price (lowest, economy class), Duration (shortest), Date (time of year which you believe is best for a wedding at this location). "
        "To make things easy, only look for one ticket, one way. "
        "You may need to make multiple searches to iteratively find the best options. "
        "You will be given no extra information, only the origin and destination. "
        "It is your job to think critically about the best options. "
        "Once you have found the best options, let the user know your shortlist of options."
    ),
)

venue_search_subagent = create_agent(
    model=model,
    tools=[venue_search],
    system_prompt=(
        "You are a wedding venue specialist. Search for the best wedding venues at the desired destination. "
        "You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria: "
        "Guest capacity (must accommodate the wedding party size), Location (scenic and convenient), "
        "Amenities (catering, outdoor space, accommodation for guests), and Reviews (highly rated). "
        "You may need to make multiple searches to iteratively find the best options. "
        "You will be given no extra information, only the destination and guest count. "
        "It is your job to think critically about the best options. "
        "Once you have found the best options, let the user know your shortlist of venues with key details."
    ),
)

playlist_search_subagent = create_agent(
    model=model,
    tools=[playlist_query],
    system_prompt=(
        "You are a music playlist specialist with access to the Chinook music database. "
        "The database has tables: Genre, Track, Artist, Album, Playlist, PlaylistTrack. "
        "Use SQL queries to find and suggest tracks for wedding playlists based on genre or artist preferences."
    ),
)

### Create the coordinator agent and its tools

In [ ]:
@tool
async def call_search_flight_subagent(runtime: ToolRuntime) -> str:
    """Use the search flight subagent to find flights from origin to destination for the wedding. No arguments needed — reads from state."""
    origin = runtime.state.get("origin", "unknown")
    destination = runtime.state.get("destination", "unknown")
    date = runtime.state.get("date", "unknown")
    query = f"Find a flight from {origin} to {destination} on {date}"
    response = await flight_search_subagent.ainvoke({"messages": [{"role": "user", "content": query}]})
    return response["messages"][-1].content

@tool
async def call_venue_search_subagent(runtime: ToolRuntime) -> str:
    """Use the search venue subagent to find wedding venues at the destination. No arguments needed — reads from state."""
    destination = runtime.state.get("destination", "unknown")
    guest_count = runtime.state.get("guest_count", "unknown")
    query = f"Find wedding venues in {destination} for {guest_count} guests"
    response = await venue_search_subagent.ainvoke({"messages": [{"role": "user", "content": query}]})
    return response["messages"][-1].content

@tool
async def call_playlist_search_subagent(runtime: ToolRuntime) -> str:
    """Use the search playlist subagent to find wedding playlist tracks matching the genre. No arguments needed — reads from state."""
    genre = runtime.state.get("genre", "unknown")
    query = f"Find tracks in the {genre} genre suitable for a wedding playlist"
    response = await playlist_search_subagent.ainvoke({"messages": [{"role": "user", "content": query}]})
    return response["messages"][-1].content

@tool
def update_wedding_details(
    origin: str,
    destination: str,
    date: str,
    guest_count: str,
    genre: str,
    runtime: ToolRuntime,
) -> Command:
    """Update the wedding details in state once the user has provided them."""
    return Command(
        update={
            "origin": origin,
            "destination": destination,
            "date": date,
            "guest_count": guest_count,
            "genre": genre,
            "messages": [ToolMessage("Successfully updated wedding details", tool_call_id=runtime.tool_call_id)],
        }
    )


agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    tools=[
        call_search_flight_subagent,
        call_venue_search_subagent,
        call_playlist_search_subagent,
        update_wedding_details,
    ],
    state_schema=WeddingState,
    system_prompt=(
        "You are a wedding planner assistant. You MUST follow these steps in order:\n"
        "Step 1: Call update_wedding_details to save the user's origin, destination, date, guest_count, and genre.\n"
        "Step 2: After state is saved, call call_search_flight_subagent, call_venue_search_subagent, "
        "and call_playlist_search_subagent. These tools require NO arguments — they automatically read "
        "from the saved state. Just call them.\n"
        "Step 3: Present a summary of the flight options, venue options, and playlist suggestions to the user.\n"
        "IMPORTANT: Never ask follow-up questions. You have all the information you need in state."
    ),
)

## Test

In [ ]:
question = (
    "I'm planning a wedding! I'm flying from Berlin to San Francisco on March 31st 2026. "
    "We're expecting 100 guests and we'd love a jazz playlist."
)

response = await agent.ainvoke({"messages": [HumanMessage(content=question)]}, config=config)

In [ ]:
from pprint import pprint

print("=== Wedding State ===")
print(f"Origin: {response.get('origin', 'N/A')}")
print(f"Destination: {response.get('destination', 'N/A')}")
print(f"Date: {response.get('date', 'N/A')}")
print(f"Guest Count: {response.get('guest_count', 'N/A')}")
print(f"Genre: {response.get('genre', 'N/A')}")
print("\n=== Agent Response ===")
print(response["messages"][-1].content)